# To Fix

For some reason, N3RP contract keeps adding eth to some of its attributes every time it is called.

In [1]:
from datetime import datetime

In [2]:
class Address:
    def __init__(self, name):
        if type(name).__name__ == "Address":
            name = name.name
        else:
            name = '0x' + name
        self.name = name
        self.address = self
    
    @property
    def str(self):
        return self.name[2:]
    
    def __add__(self, addr):
        if type(addr).__name__ == "Address":
            addr = addr.str
        self.name += addr
        return self
    
    def __iadd__(self, addr):
        self.__add__(addr)
    
    def __repr__(self):
        return f"Address({self.name})"
        
class Eth:
    def __init__(self, amount):
        self.amount = amount
    
    def __add__(self, eth):
        self.amount += eth.amount
        return self
    
    def __iadd__(self, eth):
        self.amount += eth.amount
        return self
    
    def __sub__(self, eth):
        self.amount -= eth.amount
        return self
    
    def __isub__(self, eth):
        self.amount -= eth.amount
        return self
    
    def __neg__(self):
        return -self.amount
    
    def __pos__(self):
        return +self.amount
    
    def __gt__(self, eth):
        return self.amount > eth.amount
    
    def __ge__(self, eth):
        return self.amount >= eth.amount
    
    def __lt__(self, eth):
        return self.amount < eth.amount
    
    def __le__(self, eth):
        return self.amount <= eth.amount
    
    def __eq__(self, eth):
        return self.amount == eth.amount
    
    def __ne__(self, eth):
        return self.amount != eth.amount
    
    def __repr__(self):
        return f"\N{greek capital letter xi}{self.amount}"


class Nft:
    def __init__(self, nftAddr: Address, nftId: int, price: Eth, data: str, contract):
        self.nftAddr = Address(nftAddr)
        self.str = self.nftAddr.str
        self.nftId = nftId
        self.price = price
        self.data = data
        self.contract = contract
    
    @property
    def contractName(self):
        #return self.contract._name
        nameLst = list(filter(lambda x: type(x[1]).__name__ == "Address", self.contract.__dict__.items()))
        assert len(nameLst) == 1, f"Could not locate name of contract {self.contract}"
        return nameLst[0][1]
    
    @property
    def contractType(self):
        return type(self.contract).__name__
    
    def __repr__(self):
        return f"Nft({self.nftAddr}, {self.price}, {self.data}, {self.contractName}, {self.contractType})"
    
    def __str__(self):
        return f"Nft({self.nftAddr})"
        
class Owner:
    def __init__(self, address: Address, *args):
        self.address = Address(address)
        self.str = self.address.str
        nfts = set(filter(lambda x: type(x).__name__ == "Nft",args))
        ethSet = set(filter(lambda x: type(x).__name__ == "Eth",args))
        self.eth = Eth(0)
        for ethEl in ethSet:
            self.eth += ethEl
        self.eventLogger = EventLogger()
        if 'LISTOFALLCONTRACTS' not in globals():
            globals()['LISTOFALLCONTRACTS'] = []
        for nft in nfts:
            self.setInitialOwner(self.address,nft)
    
    @property
    def assets(self):
        return set(self.nfts,self.eth)
    
    @property
    def log(self):
        return self.eventLogger.log
    
    @property
    def nfts(self):
        nftStringsList = []
        #for contractType in globals()['LISTOFALLCONTRACTS']:
        #    contracts = [k for k,v in globals().items() if type(v).__name__ == contractType]
        #    for contract in contracts:
        #        nftStringsList += [k for k,v in globals()[contract]._owners.items() if v == self.address]
        #contracts = [k for k,v in globals().items() if type(v).__name__ == "ERC721"]
        contracts = globals()['DICTIONARYOFALLCONTRACTS']['ERC721']
        for contract in contracts:
            #nftStringsList += [k for k,v in globals()[contract]._owners.items() if v == self.address]
            nftStringsList += [k for k,v in contract._owners.items() if v == self.address]
        
        allNfts = [globals()[k] for k,v in globals().items() if type(v).__name__ == 'Nft']
        myNfts = list(set([_ for _ in allNfts if _.nftAddr in nftStringsList]))
        return myNfts
    
    def __add__(self, thing):
        if type(thing).__name__ == 'Nft':
            nft = thing
            currOwnerAddr = self.lookupNftOwner(nft)
            self.approve(currOwnerAddr,nft)
            self.transferFrom(currOwnerAddr,self.address,nft)
        elif type(thing).__name__ == "Eth":
            eth = thing
            self.eth += eth
    
    def __iadd__(self, thing):
        self.__add__(thing)
    
    def __isub__(self, thing):
        self.__sub__(thing)
    
    def lookupNftOwner(self, nft):
        nftStringsList = []
        #for contractType in globals()['LISTOFALLCONTRACTS']:
        #    contracts = [k for k,v in globals().items() if type(v).__name__ == contractType]
        #    for contract in contracts:
        #        owner = globals()[contract].ownerOf(nft.nftAddr)
        #        if owner:
        #            return owner
        #contracts = [k for k,v in globals().items() if type(v).__name__ == "ERC721"]
        contracts = globals()['DICTIONARYOFALLCONTRACTS']['ERC721']
        for contract in contracts:
            #owner = globals()[contract].ownerOf(nft.nftAddr)
            owner = contract.ownerOf(nft.nftAddr)
            if owner:
                return owner
        return None
    
    def __sub__(self, thing):
        if type(thing).__name__ == 'Nft':
            nft = thing
            self.burn(nft)
        elif type(thing).__name__ == "Eth":
            eth = thing
            self.eth -= eth
    
    def __contains__(self, asset):
        return asset in self.assets
    
    def __repr__(self):
        return f"Owner({self.address},{self.assets})"
    
    def __str__(self):
        return f"Owner({self.address})"
    
    def emit(self, *args):
        self.eventLogger.emit(*args)
    
    def approve(self, to, nft):
        self.emit('Approval',self.address,to.address,nft.contractType,nft.contractName)
        #theContract = self.getContract(nft)
        #globals()[theContract].approve(to.address, nft.nftAddr, self.address)
        self.getContract(nft).approve(to.address, nft.nftAddr, self.address)
    
    def transferFrom(self,addrFrom,to,nft):
        #self.eventLogger.emit('Transfer', addrFrom, to, nftAddr)
        self.emit('Transfer', addrFrom.address, to.address, nft.nftAddr)
        #theContract = self.getContract(nft)
        #globals()[theContract].transferFrom(addrFrom.address,to.address,nft.nftAddr,self.address)
        self.getContract(nft).transferFrom(addrFrom.address,to.address,nft.nftAddr,self.address)
    
    def getContract(self, nft):
        #contractKeys = [k for k,v in dict(globals()).items() if type(v).__name__ == nft.contractType]
        #assert len(contractKeys) == 1, "No ERC721 contracts have been created. Or more than one has been created."
        #theContract = list(filter(lambda x: globals()[x]._name == nft.contractName, contractKeys))[0]
        #return theContract
        contractKeys = globals()['DICTIONARYOFALLCONTRACTS'][nft.contractType]
        assert len(contractKeys) == 1, "No ERC721 contracts have been created. Or more than one has been created."
        contractKeys = [_ for _ in contractKeys if _._name == nft.contractName]
        assert len(contractKeys) == 1, "No ERC721 contracts have been created. Or more than one has been created."
        theContract = contractKeys[0]
        return theContract
    
    def setInitialOwner(self, to, nft):
        #self.eventLogger.emit('Transfer',to,nftAddr,ERC721Name)
        self.emit('SetInitialOwner',to.address,nft.nftAddr,nft.contractType,nft.contractName)
        #theContract = self.getContract(nft)
        #globals()[theContract].setInitialOwner(to.address,nft.nftAddr,self.address)
        self.getContract(nft).setInitialOwner(to.address,nft.nftAddr,self.address)
    
    def setInitialOwners(self, toList, nfts):
        #self.eventLogger.emit('Transfer',to,nftAddr,ERC721Name)
        nftAddrs = [_.nftAddr for _ in nfts]
        contractTypes = [_.contractType for _ in nfts]
        contractNames = [_.contractNames for _ in nfts]
        self.emit('SetInitialOwners',toList,nftAddrs,nftContracts,nftNames)
        theContracts = [self.getContract(_) for _ in contractNames]
        for theContract in list(set(theContracts)):
            contractIdxs = np.where(theContracts == theContract)
            currToList = np.array(toList)[contractIdxs]
            nftAddrs = np.array(nftAddrs)[contractIdxs]
            #globals()[theContract].setInitialOwners(toList,nftAddrs,self.address)
            theContract.setInitialOwners(toList,nftAddrs,self.address)
    
    def burn(self,nft):
        self.emit('Burn',nft.nftAddr,nft.contractType,nft.contractName)
        theContract = self.getContract(nft)
        globals()[theContract]._burn(nft.nftAddr,self.address)
    
    def N3RP(self, lenderAddr: Address, borrowerAddr: Address, nftAddr: Address, basePayment: Eth, collateral: Eth, rentalDueDate: str, collateralPayoutPeriod: str):
        if 'N3RPCONTRACTBANK' not in globals().keys():
            globals()['N3RPCONTRACTBANK'] = {}
        currNumOfN3RPContracts = len(globals()["N3RPCONTRACTBANK"])
        globals()[f'N3RPNUMBER{currNumOfN3RPContracts}'] = N3RP(lenderAddr, borrowerAddr, self.address, nftAddr, basePayment, collateral, rentalDueDate, collateralPayoutPeriod)
        globals()['N3RPCONTRACTBANK'][f'N3RPNUMBER{currNumOfN3RPContracts}'] = globals()[f'N3RPNUMBER{currNumOfN3RPContracts}']
    
    def depositNft(self, N3RPContract: N3RP):
        N3RPContract.depositNft(N3RPContract.nftAddr, self.address)
        self.emit('DepositNft',self,N3RPContract.nftAddr,N3RPContract.nftContractType,N3RPContract.nftContractName)
        self.transferFrom(N3RPContract.lenderAddr, N3RPContract.address, N3RPContract.nft)
    
    def depositEth(self, N3RPContract: N3RP):
        assert self.eth > N3RPContract.basePayment + N3RPContract.collateral, f"Eth Deposit Error: your {self.eth} is insufficient for this contract requiring {N3RP.basePayment + N3RP.collateral}"
        self.eth -= N3RPContract.basePayment + N3RPContract.collateral
        self.emit('DepositEth',self,N3RPContract.nftAddr,N3RPContract.nftContractType,N3RPContract.nftContractName)
        N3RPContract.depositEth(N3RPContract.basePayment, N3RPContract.collateral, self.address)


class EventLogger:
    def __init__(self):
        self.log = {}
        self.activeClasses = []
    
    def __iadd__(self, c):
        if str(c) not in self.activeClasses:
            self.activeClasses.append(str(c))
    
    @property
    def events(self):
        return list(self.log.keys())
    
    def emit(self,eventName,eventCaller,eventResponder,*args):
        #time, #type of event, #parameters
        currTime = datetime.now().strftime('%Y/%M/%d %H:%M:%S')
        result = [eventName, eventCaller, eventResponder, currTime, *args]
        if eventName not in self.log.keys():
            self.log[eventName] = [result]
        else:
            self.log[eventName] += [result]
    
    def __repr__(self):
        return f"EventLogger({self.log},{self.activeClasses})"

class Contract:
    def __init__(self,contractTypeName,contract):
        if 'LISTOFALLCONTRACTTYPES' not in globals():
            globals()['LISTOFALLCONTRACTTYPES'] = [contractTypeName]
        else:
            globals()['LISTOFALLCONTRACTTYPES'] += [contractTypeName]
        if 'DICTIONARYOFALLCONTRACTS' not in globals():
            globals()['DICTIONARYOFALLCONTRACTS'] = {}
        if contractTypeName not in globals()['DICTIONARYOFALLCONTRACTS'].keys():
            globals()['DICTIONARYOFALLCONTRACTS'][contractTypeName] = [contract]
        if contract not in globals()['DICTIONARYOFALLCONTRACTS'][contractTypeName]:
            globals()['DICTIONARYOFALLCONTRACTS'][contractTypeName] += [contract]
    
    @staticmethod
    def contractTypes():
        return globals()['LISTOFALLCONTRACTTYPES']
    
    @staticmethod
    def contractDict():
        return globals()['DICTIONARYOFALLCONTRACTS']


class ERC721(Contract):
    def __init__(self, name_: Address, symbol_: str):
        self._name = name_
        self._symbol = symbol_
        self._owners = {}
        self._balances = {}
        self._tokenApprovals = {}
        self._operatorApprovals = {}
        self.eventLogger = EventLogger()
        self.allTokensAssigned = False
        self.nftsRemainingToAssign = 10000
        super().__init__(type(self).__name__, self)
    
    @property
    def log(self):
        return self.eventLogger.log
    
    def __repr__(self):
        return f"ERC721({self._name}, {self._symbol}, {self._owners}, {self._balances}, {self._tokenApprovals}, {self._operatorApprovals}, {self.eventLogger}, {self.allTokensAssigned})"
    
    def __str__(self):
        return f"ERC721({self._name})"
    
    def _exists(self, nftAddr):
        return nftAddr in self._owners.keys()
    
    def balanceOf(self, ownerAddress):
        if ownerAddress in self._balances.keys():
            return self._balances[ownerAddress]
        return None
    
    def ownerOf(self, nftAddr):
        if self._exists(nftAddr):
            return self._owners[nftAddr]
        return None
    
    def _approve(self, to, nftAddr):
        self._tokenApprovals[nftAddr] = to
        #self.eventLogger.emit("Approval",self.ownerOf(nftAddr), to, nftAddr)
        self.emit("Approval",self.ownerOf(nftAddr), to, nftAddr)
    
    def approve(self, to, nftAddr, msgSender):
        owner = self.ownerOf(nftAddr)
        assert owner != to, "ERC721: approve caller is not owner nor approved for all"
        #assert msgSender == owner | isApprovedForAll(owner, msgSender), "Error"
        self._approve(to, nftAddr)
    
    def isApprovedForAll(self,owner,operator):
        if owner in self._operatorApprovals.keys():
            return operator in self._operatorApprovals[owner].keys()
        return False
    
    def transferFrom(self,addrFrom,to,nftAddr,msgSender):
        assert self._isApprovedOrOwner(msgSender, nftAddr), "ERC721: transfer caller is not owner nor approved"
        self._transfer(addrFrom, to, nftAddr)
    
    def _isApprovedOrOwner(self, spender, nftAddr):
        assert self._exists(nftAddr), "ERC721: operator query for nonexistent token"
        owner = self.ownerOf(nftAddr)
        print(f'{spender=}, {nftAddr=}, {owner=}')
        print((spender == owner), (self.getApproved(nftAddr) == spender))
        return (spender == owner) | (self.getApproved(nftAddr) == spender) | (self.isApprovedForAll(owner, spender))
    
    def getApproved(self, nftAddr):
        assert self._exists(nftAddr), "ERC721: approved query for nonexistent token"
        return nftAddr in self._tokenApprovals.keys();
    
    def _transfer(self, addrFrom, to, nftAddr):
        assert self.ownerOf(nftAddr) == addrFrom, "ERC721: transfer from incorrect owner"
        assert to != None, "ERC721: transfer to the zero address"
        self._beforeTokenTransfer(addrFrom, to, nftAddr)
        self._approve(None, nftAddr)
        self._balances[addrFrom] -= 1
        if to not in self._balances.keys():
            self._balances[to] = 0
        self._balances[to] += 1
        self._owners[nftAddr] = to
        #self.eventLogger.emit("Transfer",addrFrom, to, nftAddr)
        self.emit("Transfer",addrFrom, to, nftAddr)
        self._afterTokenTransfer(addrFrom, to, nftAddr)
    
    def _afterTokenTransfer(self, addrFrom, to, nftAddr):
        pass
    
    def _safeMint(self, to, nftAddr, _data=""):
        self._mint(to, nftAddr)
        assert self._checkOnERC721Received(None, to, nftAddr, _data), "ERC721: transfer to non ERC721Receiver implementer"
    
    def _checkOnERC721Received(self, addrFrom, to, nftAddr, _data):
        #Add in later
        return True
    
    def _mint(self, to, nftAddr):
        assert to != None, "ERC721: mint to the zero address"
        assert not self._exists(nftAddr), "ERC721: token already minted"
        self._beforeTokenTransfer(None, to, nftAddr)
        if to not in self._balances.keys():
            self._balances[to] = 0
        self._balances[to] += 1
        self._owners[nftAddr] = to
        #self.eventLogger.emit("Transfer",None, to, nftAddr)
        self.emit("Transfer",None, to, nftAddr)
        self._afterTokenTransfer(None, to, nftAddr)
    
    def _beforeTokenTransfer(self, addrFrom, to, nftAddr):
        pass
    
    def _afterTokenTransfer(self, addrFrom, to, nftAddr):
        pass
    
    def setInitialOwner(self, to, nftAddr, msgSender):
        self.emit("Mint",msgSender,to,nftAddr)
        self._mint(to, nftAddr)
    
    def setInitialOwners(self, toList, nftAddrs, msgSender):
        [self.setInitialOwner(to, nftAddr, msgSender) for to,nftAddr in zip(toList,nftAddrs)]
    
    def emit(self,*args):
        self.eventLogger.emit(*args)
    
    def _burn(self, nftAddr, msgSender):
        owner = self.ownerOf(nftAddr)
        self._beforeTokenTransfer(owner, None, nftAddr)
        
        self._approve(None, nftAddr)
        
        self._balances[owner] -= 1
        del self._owners[nftAddr]
        
        self.emit("Transfer", msgSender, owner, None, nftAddr)
        
        self._afterTokenTransfer(owner, None, nftAddr)



class N3RP(Contract):
    def __init__(self,
                 lenderAddr: Address,
                 borrowerAddr: Address,
                 initiatorAddr: Address,
                 nft: Nft,
                 basePayment: Eth,
                 collateral: Eth,
                 rentalDueDate: str, #Will be a ethereum blocktime, in reality
                 collateralPayoutPeriod: str, #Will be an integer number of blocks, in reality
                ):
        self.address = Address(type(self).__name__) + nft.nftAddr
        self.eth = Eth(0)
        self.str = self.address
        self.lenderAddr = lenderAddr
        self.borrowerAddr = borrowerAddr
        self.initiatorAddr = initiatorAddr
        self.nft = nft
        self.basePayment = basePayment
        self.collateral = collateral
        self.rentalDueDate = rentalDueDate
        self.collateralPayoutPeriod = collateralPayoutPeriod
        super().__init__(type(self).__name__,self)
    
    @property
    def nftAddr(self):
        return self.nft.nftAddr
    
    @property
    def nftContractName(self):
        return self.nft.contractName
    
    @property
    def nftContractType(self):
        return self.nft.contractType
    
    @property
    def nftIsDeposited(self):
        #theContract = self.getContract(self.nft)
        #if self.address == globals()[theContract].ownerOf(self.nft.nftAddr):
        if self.address == self.getContract(self.nft).ownerOf(self.nft.nftAddr):
            return True
        return False
    
    @property
    def ethIsDeposited(self):
        return self.basePayment + self.collateral >= self.eth
    
    def getContract(self, nft):
        #contractKeys = [k for k,v in dict(globals()).items() if type(v).__name__ == nft.contractType]
        contractKeys = globals()['DICTIONARYOFALLCONTRACTS'][nft.contractType]
        assert len(contractKeys) == 1, "No ERC721 contracts have been created. Or more than one has been created."
        #contractKeys = list(filter(lambda x: globals()[x]._name == nft.contractName, contractKeys))
        contractKeys = [_ for _ in contractKeys if _._name == nft.contractName]
        assert len(contractKeys) == 1, "No ERC721 contracts have been created. Or more than one has been created."
        theContract = contractKeys[0]
        return theContract
    
    def depositNft(self, nftAddr: Address, msgSender: Address):
        assert self.nftIsDeposited is False, f"Nft Deposit Error: an nft has already been deposited here"
        assert self.lenderAddr == msgSender, f"Nft Deposit Error: sender {msgSender} is not lender {self.lenderAddr}"
        assert self.nftAddr == nftAddr, f"Nft Deposit Error: deposited nft {nftAddr} does not match the stipulated nft {self.nftAddr}"
        #self.nftIsDeposited = True
        return True
    
    def depositEth(self, basePayment: Eth, collateral: Eth, depositorAddr: Address):
        assert self.ethIsDeposited is False, f"Eth Deposit Error: eth has already been deposited here"
        assert self.basePayment == basePayment, f"Eth Deposit Error: a base payment of {self.basePayment} is required, but {basePayment} was received"
        assert self.collateral == collateral, f"Eth Deposit Error: a collateral of {self.collateral} is required, but {collateral} was received"
        assert self.depositorAddr == depositorAddr, f"Eth Deposit Error: sender {depositorAddr} is not the depositor {self.depositor}"
        self.eth = self.basePayment + self.depositorAddr
        return True
    
    def __repr__(self):
        return f"N3RP({self.address},{self.eth},{self.str},{self.lenderAddr},{self.borrowerAddr},{self.initiatorAddr},{self.nft},{self.basePayment},{self.collateral},{self.rentalDueDate},{self.collateralPayoutPeriod},{self.nftIsDeposited},{self.ethIsDeposited})"


In [3]:
theERC721 = ERC721(Address('CryptoPunks'), 'CRYPTOPUNKS')

nft1 = Nft(Address('JjCoin'), 1, Eth(100), '', theERC721)
nft2 = Nft(Address('GgCoin'), 2, Eth(1000), '', theERC721)
owner1 = Owner('Gg')
owner2 = Owner('Jj')
owner3 = Owner('xXxCryptoLordxXx')
# owner3.approve(owner2.address,nft2.nftId,'CryptoPunks')

In [4]:
nft1.nftAddr
# owner2.setInitialOwner(owner2, nft1)
owner1 + Eth(100)

In [5]:
lender = Owner(Address('Lender'))
lender.setInitialOwner(lender, nft1)
borrower = Owner(Address('Borrower'))
borrower + Eth(1000)

In [6]:
# lenderAddr, borrowerAddr, nftAddr, basePayment, collateral, rentalDueDate
lender.N3RP(lender.address, borrower.address, nft1, Eth(2), Eth(25), '20220209', '20220211')

In [7]:
N3RPContractStr = list(globals()['N3RPCONTRACTBANK'].keys())[0]
lender.depositNft(globals()[N3RPContractStr])

spender=Address(0xLender), nftAddr=Address(0xJjCoin), owner=Address(0xLender)
True False


In [8]:
N3RPContractStr

'N3RPNUMBER0'

In [9]:
theERC721.ownerOf(globals()[N3RPContractStr].nft.nftAddr)

Address(0xN3RPJjCoin)

In [10]:
nft1

Nft(Address(0xJjCoin), Ξ100, , Address(0xCryptoPunks), ERC721)

In [11]:
theERC721

ERC721(Address(0xCryptoPunks), CRYPTOPUNKS, {Address(0xJjCoin): Address(0xN3RPJjCoin)}, {Address(0xLender): 0, Address(0xN3RPJjCoin): 1}, {Address(0xJjCoin): None}, {}, EventLogger({'Mint': [['Mint', Address(0xLender), Address(0xLender), '2022/43/06 00:43:38', Address(0xJjCoin)]], 'Transfer': [['Transfer', None, Address(0xLender), '2022/43/06 00:43:38', Address(0xJjCoin)], ['Transfer', Address(0xLender), Address(0xN3RPJjCoin), '2022/43/06 00:43:38', Address(0xJjCoin)]], 'Approval': [['Approval', Address(0xLender), None, '2022/43/06 00:43:38', Address(0xJjCoin)]]},[]), False)

In [12]:
borrower.nfts

[]

In [13]:
globals()[N3RPContractStr].__dict__

{'address': Address(0xN3RPJjCoin),
 'eth': Ξ0,
 'str': Address(0xN3RPJjCoin),
 'lenderAddr': Address(0xLender),
 'borrowerAddr': Address(0xBorrower),
 'initiatorAddr': Address(0xLender),
 'nft': Nft(Address(0xJjCoin), Ξ100, , Address(0xCryptoPunks), ERC721),
 'basePayment': Ξ2,
 'collateral': Ξ25,
 'rentalDueDate': '20220209',
 'collateralPayoutPeriod': '20220211'}

In [14]:
owner1.depositEth(N3RPNUMBER0)

AssertionError: Eth Deposit Error: eth has already been deposited here

In [15]:
owner1.eth

Ξ48

In [16]:
globals()[N3RPContractStr]

N3RP(Address(0xN3RPJjCoin),Ξ0,Address(0xN3RPJjCoin),Address(0xLender),Address(0xBorrower),Address(0xLender),Nft(Address(0xJjCoin)),Ξ77,Ξ25,20220209,20220211,True,True)

In [26]:
list(Contract.contractDict().values())[1][0]

N3RP(Address(0xN3RPJjCoin),Ξ0,Address(0xN3RPJjCoin),Address(0xLender),Address(0xBorrower),Address(0xLender),Nft(Address(0xJjCoin)),Ξ602,Ξ25,20220209,20220211,True,True)

In [24]:
from pprint import pprint

pprint(Contract.contractDict())

{'ERC721': [ERC721(Address(0xCryptoPunks), CRYPTOPUNKS, {Address(0xJjCoin): Address(0xN3RPJjCoin)}, {Address(0xLender): 0, Address(0xN3RPJjCoin): 1}, {Address(0xJjCoin): None}, {}, EventLogger({'Mint': [['Mint', Address(0xLender), Address(0xLender), '2022/43/06 00:43:38', Address(0xJjCoin)]], 'Transfer': [['Transfer', None, Address(0xLender), '2022/43/06 00:43:38', Address(0xJjCoin)], ['Transfer', Address(0xLender), Address(0xN3RPJjCoin), '2022/43/06 00:43:38', Address(0xJjCoin)]], 'Approval': [['Approval', Address(0xLender), None, '2022/43/06 00:43:38', Address(0xJjCoin)]]},[]), False)],
 'N3RP': [N3RP(Address(0xN3RPJjCoin),Ξ0,Address(0xN3RPJjCoin),Address(0xLender),Address(0xBorrower),Address(0xLender),Nft(Address(0xJjCoin)),Ξ552,Ξ25,20220209,20220211,True,True)]}


In [19]:
list(Contract.contractDict().values())[0][0] == list(Contract.contractDict().values())[0][1]

IndexError: list index out of range